# Data Fusion Contest 2026 - Задача 2 "Киберполка"

Участникам необходимо решить задачу multi-label классификации для 41 финансового продукта клиентов банка на основе обезличенных данных заранее предоставленных признаков.

## Постановка задачи

Клиенты банка владеют разными банковскими продуктами — различными видами счетов, карт и услуг. Одним из запросов бизнеса является желание ранжировать эти продукты по вероятности их открытия клиентами.

В данной задаче продуктов “на полке” 41 штука. В отличие от классических задач рекомендаций, бизнес интересуют вероятности открытия каждого из продуктов. Умея хорошо предсказывать эти вероятности, бизнес имел бы возможность гибко настраивать рекомендации.

В этом соревновании участникам предстоит работать с полностью анонимными и обфусцированными данными 1,000,0000 клиентов:

- Описания банковских продуктов не передаются. Также не передаются и описания признаков. Для признаков передается только информация об их исходном типе (категорийные признаки cat_feature_i и числовые признаки num_feature_j).
- Признаков достаточно много: основной набор в 200 признаков, а также дополнительный набор с более 2,000 признаками.
- В признаках много пропущенных значений, а также присутствуют выбросы.
- 750,000 клиентов имеют разметку и составляют тренировочные данные. Оставшиеся 250,000 клиентов составляют тестовые данные.

Подробнее про структуру данных можно узнать на странице “Данные”.

## Формат решений

Это соревнование с разметкой предоставленного вам .parquet файла. Вам необходимо создать алгоритм, способный по предоставленным в рамках соревнования данным, создать новый .parquet файл с 42 столбцами:

```text
customer_id, predict_1_1, predict_1_2, ... , predict_10_1
1750000, -4.921889, -5.700829, ... , -0.954659
1750001, -4.963202, -6.826517, ... , -0.622011
...
1999999, -4.249957, -4.785856, ... , -0.931220
```

- customer_id – идентификатор клиента;
- predict_i – предсказание вашего алгоритма для класса target_i. Например predict_1_1 для target_1_1 и т.д.

Предсказания необходимо построить для всех 250,000 клиентов в тестовых данных.

Пример sample_submit.parquet доступен на странице “Данные”.

## Проверка решений

Решения проверяются автоматически путем сопоставления с известными истинными историческими значениями суммарных переводов клиентов банка со своих счетов. Истинные исторические значения в тестовых данных доступны только организаторам.

Метрика соревнования — Macro Averaged ROC-AUC. Для multi-label это эквивалентно простому усреднению ROC-AUC по каждому классу.

Для расчета метрики используется sklearn имплементация:

```python
from sklearn.metrics import roc_auc_score

roc_auc_score(y_true, y_pred, average="macro")
```

Соотношение public/private в соревновании составляет 30/70:

- 75,000 (30%) клиентских записей используются для результатов на public лидерборде.
- 175,000 (70%) клиентских записей  используются для результатов на private лидерборде.

Победители соревнования определяются по результатам на private лидерборде.
Для private лидерборда можно выбрать до 2 финальных решений.


## Данные

Для решения задачи 2 “Киберполка” предлагается два наборов данных, а также пример решения задачи.

**Важные напоминания:**

- Расшифровка названий признаков, как и расшифровка названий целевых переменных, не предоставляется.
- Решения принимаются в формате .parquet. Форма файлов, названия и порядок столбцов должны строго соответствовать таковым в sample_submit.parquet.

### Основные материалы соревнования

- `train_target.parquet` ? 4.8MB, разметка целевых переменных для тренировочных данных
- `sample_submit.parquet` ? 43.9MB, пример базового решения на основе Catboost
- `baseline_catboost.ipynb` ? 231.5KB, ноутбук с реализацией базового решения на основе Catboost

### Тренировочные данные

Для обучения моделей участникам представляются 2 файла с различными группами признаков для моделирования.
В качестве ключа для объединения данных выступает идентификатор клиента customer_id.

- `train_main_features.parquet` ? 119.0MB, основная группа признаков (категорийные и числовые)
- `train_extra_features.parquet` ? 959.3MB, дополнительные признаки для решения задачи (числовые признаки)

### Тестовые данные

Аналогичные 2 файла, необходимые для подготовки ваших предсказаний:

- `test_main_features.parquet` ? 42.4MB, основная группа признаков (категорийные и числовые)
- `test_extra_features.parquet` ? 320.4MB, дополнительные признаки для решения задачи (числовые признаки)


## 1. Imports and utility functions

This section initializes libraries, memory-safe helpers, and streaming DSV builders for Kaggle.


In [1]:
import gc
import csv
from itertools import zip_longest
from pathlib import Path

import pandas as pd
import pyarrow.dataset as ds
import pyarrow.parquet as pq
from catboost import Pool, CatBoostClassifier

DATA_DIR = Path('/kaggle/input/datasets/yngbogdnn27/fusion2')
WORK_DIR = Path('/kaggle/working/catboost_cache')
WORK_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_DSV = WORK_DIR / 'train_full.dsv'
TEST_DSV = WORK_DIR / 'test_full.dsv'
TRAIN_CD = WORK_DIR / 'train.cd'
TEST_CD = WORK_DIR / 'test.cd'

BATCH_ROWS = 1000  # smaller value => lower RAM peak, slower preprocessing


def write_cd(cd_path: Path, label_count: int, feature_cols, cat_feature_names):
    lines = []

    for i in range(label_count):
        lines.append(f'{i}\tLabel\n')

    feature_offset = label_count
    cat_set = set(cat_feature_names)
    for j, feature_name in enumerate(feature_cols):
        if feature_name in cat_set:
            col_idx = feature_offset + j
            lines.append(f'{col_idx}\tCateg\t{feature_name}\n')

    cd_path.write_text(''.join(lines), encoding='utf-8')


def _safe_zip(*iterables):
    sentinel = object()
    for items in zip_longest(*iterables, fillvalue=sentinel):
        if any(x is sentinel for x in items):
            raise ValueError('Batch streams are misaligned (different number of batches).')
        yield items


def _append_batch_to_dsv(df: pd.DataFrame, path: Path):
    df.to_csv(
        path,
        sep='\t',
        header=False,
        index=False,
        mode='a',
        na_rep='nan',
        quoting=csv.QUOTE_MINIMAL,
        float_format='%.7g',
    )


def build_catboost_dsv(
    main_path: Path,
    extra_path: Path,
    output_path: Path,
    feature_main_cols,
    extra_cols_to_add,
    cat_feature_names,
    key: str = 'customer_id',
    batch_rows: int = BATCH_ROWS,
    target_path: Path = None,
    target_cols=None,
):
    if output_path.exists():
        output_path.unlink()

    main_schema_cols = pq.ParquetFile(main_path).schema.names
    extra_schema_cols = pq.ParquetFile(extra_path).schema.names

    main_scan_cols = list(feature_main_cols)
    extra_scan_cols = list(extra_cols_to_add)

    check_key = key in main_schema_cols and key in extra_schema_cols
    if check_key:
        if key not in main_scan_cols:
            main_scan_cols.append(key)
        if key not in extra_scan_cols:
            extra_scan_cols.append(key)

    main_batches = ds.dataset(main_path, format='parquet').scanner(
        columns=main_scan_cols,
        batch_size=batch_rows,
        use_threads=True,
    ).to_batches()

    extra_batches = ds.dataset(extra_path, format='parquet').scanner(
        columns=extra_scan_cols,
        batch_size=batch_rows,
        use_threads=True,
    ).to_batches()

    if target_path is not None:
        if not target_cols:
            raise ValueError('target_cols must be provided when target_path is set.')

        target_batches = ds.dataset(target_path, format='parquet').scanner(
            columns=list(target_cols),
            batch_size=batch_rows,
            use_threads=True,
        ).to_batches()

        iterator = _safe_zip(main_batches, extra_batches, target_batches)
    else:
        iterator = _safe_zip(main_batches, extra_batches)

    for batch_idx, packed in enumerate(iterator, start=1):
        if target_path is not None:
            main_batch, extra_batch, target_batch = packed
        else:
            main_batch, extra_batch = packed
            target_batch = None

        main_df = main_batch.to_pandas()
        extra_df = extra_batch.to_pandas()

        if check_key:
            if not main_df[key].equals(extra_df[key]):
                raise ValueError(f'Rows are misaligned by {key} in batch {batch_idx}.')

        if key in main_df.columns:
            main_df.drop(columns=[key], inplace=True)
        if key in extra_df.columns:
            extra_df.drop(columns=[key], inplace=True)

        feature_df = pd.concat(
            [main_df[feature_main_cols], extra_df[extra_cols_to_add]],
            axis=1,
            copy=False,
        )

        if target_batch is not None:
            target_df = target_batch.to_pandas()
            chunk_df = pd.concat([target_df[list(target_cols)], feature_df], axis=1, copy=False)
            del target_df
        else:
            chunk_df = feature_df

        _append_batch_to_dsv(chunk_df, output_path)

        del main_df, extra_df, feature_df, chunk_df
        gc.collect()

        if batch_idx % 20 == 0:
            print(f'Processed {batch_idx} batches -> {output_path.name}')

    gc.collect()


## 2. Dataset paths and schema checks

Define all input paths, infer feature/target schema, and validate train/test consistency.


In [2]:
train_main_path = DATA_DIR / 'train_main_features.parquet'
train_extra_path = DATA_DIR / 'train_extra_features.parquet'
test_main_path = DATA_DIR / 'test_main_features.parquet'
test_extra_path = DATA_DIR / 'test_extra_features.parquet'
target_path = DATA_DIR / 'train_target.parquet'

train_main_cols = pq.ParquetFile(train_main_path).schema.names
train_extra_cols = pq.ParquetFile(train_extra_path).schema.names
test_main_cols = pq.ParquetFile(test_main_path).schema.names
test_extra_cols = pq.ParquetFile(test_extra_path).schema.names
target_cols = [c for c in pq.ParquetFile(target_path).schema.names if c != 'customer_id']

feature_main_cols = [c for c in train_main_cols if c != 'customer_id']
extra_cols_to_add = [c for c in train_extra_cols if c not in train_main_cols and c != 'customer_id']
feature_cols = feature_main_cols + extra_cols_to_add
cat_feature_names = [c for c in feature_cols if c.startswith('cat_feature')]

# Keep train/test feature layout identical.
test_feature_main_cols = [c for c in test_main_cols if c != 'customer_id']
test_extra_cols_to_add = [c for c in test_extra_cols if c not in test_main_cols and c != 'customer_id']
test_feature_cols = test_feature_main_cols + test_extra_cols_to_add

if feature_cols != test_feature_cols:
    raise ValueError('Train/Test feature layouts differ. Cannot build a consistent pool.')

write_cd(TRAIN_CD, label_count=len(target_cols), feature_cols=feature_cols, cat_feature_names=cat_feature_names)
write_cd(TEST_CD, label_count=0, feature_cols=feature_cols, cat_feature_names=cat_feature_names)

print('Target columns:', len(target_cols))
print('Feature columns:', len(feature_cols))
print('Categorical features:', len(cat_feature_names))


Target columns: 41
Feature columns: 2440
Categorical features: 67


## 3. Build training artifacts (streaming)

Create a memory-safe training DSV file from Parquet sources.


In [3]:
build_catboost_dsv(
    main_path=train_main_path,
    extra_path=train_extra_path,
    output_path=TRAIN_DSV,
    feature_main_cols=feature_main_cols,
    extra_cols_to_add=extra_cols_to_add,
    cat_feature_names=cat_feature_names,
    key='customer_id',
    batch_rows=BATCH_ROWS,
    target_path=target_path,
    target_cols=target_cols,
)

print('Built:', TRAIN_DSV)


Processed 20 batches -> train_full.dsv
Processed 40 batches -> train_full.dsv
Processed 60 batches -> train_full.dsv
Processed 80 batches -> train_full.dsv
Processed 100 batches -> train_full.dsv
Processed 120 batches -> train_full.dsv
Processed 140 batches -> train_full.dsv
Processed 160 batches -> train_full.dsv
Processed 180 batches -> train_full.dsv
Processed 200 batches -> train_full.dsv
Processed 220 batches -> train_full.dsv
Processed 240 batches -> train_full.dsv
Processed 260 batches -> train_full.dsv
Processed 280 batches -> train_full.dsv
Processed 300 batches -> train_full.dsv
Processed 320 batches -> train_full.dsv
Processed 340 batches -> train_full.dsv
Processed 360 batches -> train_full.dsv
Processed 380 batches -> train_full.dsv
Processed 400 batches -> train_full.dsv
Processed 420 batches -> train_full.dsv
Processed 440 batches -> train_full.dsv
Processed 460 batches -> train_full.dsv
Processed 480 batches -> train_full.dsv
Processed 500 batches -> train_full.dsv
Proc

## 4. Inspect training artifacts

Print generated training file paths and sizes.


In [4]:
print('Train DSV path:', TRAIN_DSV)
print('Train CD path:', TRAIN_CD)
print('Train DSV size (GB):', round(TRAIN_DSV.stat().st_size / (1024 ** 3), 3))


Train DSV path: /kaggle/working/catboost_cache/train_full.dsv
Train CD path: /kaggle/working/catboost_cache/train.cd
Train DSV size (GB): 11.385


## 5. Build test artifacts (streaming)

Create the test DSV file with the same feature layout.


In [5]:
build_catboost_dsv(
    main_path=test_main_path,
    extra_path=test_extra_path,
    output_path=TEST_DSV,
    feature_main_cols=feature_main_cols,
    extra_cols_to_add=extra_cols_to_add,
    cat_feature_names=cat_feature_names,
    key='customer_id',
    batch_rows=BATCH_ROWS,
)

print('Built:', TEST_DSV)


Processed 20 batches -> test_full.dsv
Processed 40 batches -> test_full.dsv
Processed 60 batches -> test_full.dsv
Processed 80 batches -> test_full.dsv
Processed 100 batches -> test_full.dsv
Processed 120 batches -> test_full.dsv
Processed 140 batches -> test_full.dsv
Processed 160 batches -> test_full.dsv
Processed 180 batches -> test_full.dsv
Processed 200 batches -> test_full.dsv
Processed 220 batches -> test_full.dsv
Processed 240 batches -> test_full.dsv
Built: /kaggle/working/catboost_cache/test_full.dsv


## 6. Inspect test artifacts

Print generated test file paths and sizes.


In [6]:
print('Test DSV path:', TEST_DSV)
print('Test CD path:', TEST_CD)
print('Test DSV size (GB):', round(TEST_DSV.stat().st_size / (1024 ** 3), 3))


Test DSV path: /kaggle/working/catboost_cache/test_full.dsv
Test CD path: /kaggle/working/catboost_cache/test.cd
Test DSV size (GB): 3.776


## 7. Train CatBoost model

Train the GPU model using file-based pools to avoid RAM spikes.


In [7]:
train_pool = Pool(
    data=str(TRAIN_DSV),
    column_description=str(TRAIN_CD),
    delimiter='\t',
    has_header=False,
    thread_count=4,
)

model = CatBoostClassifier(
    iterations=1000,
    depth=6,
    learning_rate=0.1,
    loss_function='MultiLogloss',
    random_seed=42,
    verbose=100,
    task_type='GPU',
    used_ram_limit='24gb',
)

model.fit(train_pool)

del train_pool
gc.collect()

0:	learn: 0.4839990	total: 10.8s	remaining: 3h 18s
100:	learn: 0.0855802	total: 3m 8s	remaining: 28m
200:	learn: 0.0833710	total: 6m 4s	remaining: 24m 7s
300:	learn: 0.0820360	total: 8m 57s	remaining: 20m 47s
400:	learn: 0.0812207	total: 11m 41s	remaining: 17m 27s
500:	learn: 0.0806008	total: 14m 21s	remaining: 14m 17s
600:	learn: 0.0801839	total: 16m 53s	remaining: 11m 12s
700:	learn: 0.0798082	total: 19m 27s	remaining: 8m 18s
800:	learn: 0.0795202	total: 21m 56s	remaining: 5m 27s
900:	learn: 0.0792676	total: 24m 23s	remaining: 2m 40s
999:	learn: 0.0790025	total: 26m 50s	remaining: 0us


0

## 8. Inference and submission export

Generate test predictions and save `submission.parquet` in the required format.


In [8]:
test_pool = Pool(
    data=str(TEST_DSV),
    column_description=str(TEST_CD),
    delimiter='\t',
    has_header=False,
    thread_count=4,
)

test_predict = model.predict(test_pool, prediction_type='RawFormulaVal')

sample_submit = pd.read_parquet(DATA_DIR / 'sample_submit.parquet')
result_df = sample_submit.copy()
result_df.iloc[:, 1:] = test_predict
result_df['customer_id'] = result_df['customer_id'].astype('int32')
result_df.to_parquet('submission.parquet', index=False)

print('Saved: submission.parquet')

Saved: submission.parquet


In [9]:
'''
Final macro ROC-AUC on public ds = 0,8516956921
At the time of submitting the solution it was 3rd place on leaderboard
Run on Kaggle w GPU T4
'''

'\nFinal macro ROC-AUC on public ds = 0,8516956921\nAt the time of submitting the solution it was 3rd place on leaderboard\nRun on Kaggle w GPU T4\n'